# Sequential DQN

In this notebook, we'll build a neural network that can learn to play games through reinforcement learning. More specifically, we'll use Q-learning to train an agent to play a game called [Cart-Pole](https://gym.openai.com/envs/CartPole-v0). In this game, a freely swinging pole is attached to a cart. The cart can move to the left and right, and the goal is to keep the pole upright as long as possible.

![Cart-Pole](assets/cart-pole.jpg)

We can simulate this game using [OpenAI Gym](https://gym.openai.com/). First, let's check out how OpenAI Gym works. Then, we'll get into training an agent to play the Cart-Pole game.

In [1]:
# In this one we should define and detect GPUs for tensorflow
# GPUs or CPU
import tensorflow as tf

# Check TensorFlow Version
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

/home/aras/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


TensorFlow Version: 1.7.1
Default GPU Device: 


>**Note:** Make sure you have OpenAI Gym cloned into the same directory with this notebook. I've included `gym` as a submodule, so you can run `git submodule --init --recursive` to pull the contents into the `gym` repo.

>**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import gym

# Create the Cart-Pole game environment
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

We interact with the simulation through `env`. To show the simulation running, you can use `env.render()` to render one frame. Passing in an action as an integer to `env.step` will generate the next step in the simulation.  You can see how many actions are possible from `env.action_space` and to get a random action you can use `env.action_space.sample()`. This is general to all Gym games. In the Cart-Pole game, there are two possible actions, moving the cart left or right. So there are two actions we can take, encoded as 0 and 1.

Run the code below to watch the simulation run.

In [3]:
import numpy as np
state = env.reset()
for _ in range(10):
    # env.render()
    action = env.action_space.sample()
    next_state, reward, done, info = env.step(action) # take a random action
    #print('state, action, next_state, reward, done, info:', state, action, next_state, reward, done, info)
    state = next_state
    if done:
        state = env.reset()

To shut the window showing the simulation, use `env.close()`.

If you ran the simulation above, we can look at the rewards:

In [4]:
# print(rewards[-20:])
# print(np.array(rewards).shape, np.array(states).shape, np.array(actions).shape, np.array(dones).shape)
# print(np.array(rewards).dtype, np.array(states).dtype, np.array(actions).dtype, np.array(dones).dtype)
# print(np.max(np.array(actions)), np.min(np.array(actions)))
# print((np.max(np.array(actions)) - np.min(np.array(actions)))+1)
# print(np.max(np.array(rewards)), np.min(np.array(rewards)))
# print(np.max(np.array(states)), np.min(np.array(states)))

The game resets after the pole has fallen past a certain angle. For each frame while the simulation is running, it returns a reward of 1.0. The longer the game runs, the more reward we get. Then, our network's goal is to maximize the reward by keeping the pole vertical. It will do this by moving the cart to the left and the right.

## Q-Network

We train our Q-learning agent using the Bellman Equation:

$$
Q(s, a) = r + \gamma \max{Q(s', a')}
$$

where $s$ is a state, $a$ is an action, and $s'$ is the next state from state $s$ and action $a$.

Before we used this equation to learn values for a Q-_table_. However, for this game there are a huge number of states available. The state has four values: the position and velocity of the cart, and the position and velocity of the pole. These are all real-valued numbers, so ignoring floating point precisions, you practically have infinite states. Instead of using a table then, we'll replace it with a neural network that will approximate the Q-table lookup function.

<img src="assets/deep-q-learning.png" width=450px>

Now, our Q value, $Q(s, a)$ is calculated by passing in a state to the network. The output will be Q-values for each available action, with fully connected hidden layers.

<img src="assets/q-network.png" width=550px>


As I showed before, we can define our targets for training as $\hat{Q}(s,a) = r + \gamma \max{Q(s', a')}$. Then we update the weights by minimizing $(\hat{Q}(s,a) - Q(s,a))^2$. 

For this Cart-Pole game, we have four inputs, one for each value in the state, and two outputs, one for each action. To get $\hat{Q}$, we'll first choose an action, then simulate the game using that action. This will get us the next state, $s'$, and the reward. With that, we can calculate $\hat{Q}$ then pass it back into the $Q$ network to run the optimizer and update the weights.

Below is my implementation of the Q-network. I used two fully connected layers with ReLU activations. Two seems to be good enough, three might be better. Feel free to try it out.

In [5]:
def model_input(state_size, lstm_size, batch_size=1):
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    targetQs = tf.placeholder(tf.float32, [None], name='targetQs')
    # RNN
    gru = tf.nn.rnn_cell.GRUCell(lstm_size)
    cell = tf.nn.rnn_cell.MultiRNNCell([gru], state_is_tuple=False)
    initial_state = cell.zero_state(batch_size, tf.float32)
    return states, actions, targetQs, cell, initial_state

In [6]:
# RNN generator or sequence generator
def generator(states, num_classes, initial_state, cell, lstm_size, reuse=False): 
    with tf.variable_scope('generator', reuse=reuse):
        # First fully connected layer
        inputs = tf.layers.dense(inputs=states, units=lstm_size)
        print(states.shape, inputs.shape)
        
        # with tf.variable_scope('dynamic_rnn_', reuse=tf.AUTO_REUSE):
        # dynamic means adapt to the batch_size
        inputs_rnn = tf.reshape(inputs, [1, -1, lstm_size]) # NxH -> 1xNxH
        print(inputs_rnn.shape, initial_state.shape)
        outputs_rnn, final_state = tf.nn.dynamic_rnn(cell=cell, inputs=inputs_rnn, initial_state=initial_state)
        print(outputs_rnn.shape, final_state.shape)
        outputs = tf.reshape(outputs_rnn, [-1, lstm_size]) # 1xNxH -> NxH
        print(outputs.shape)

        # Last fully connected layer
        logits = tf.layers.dense(inputs=outputs, units=num_classes)
        print(logits.shape)
        #predictions = tf.nn.softmax(logits)
        
        # logits are the action logits
        return logits, final_state

In [7]:
def model_loss(action_size, hidden_size, states, cell, initial_state, actions, targetQs):
    actions_logits, final_state = generator(states=states, cell=cell, initial_state=initial_state, 
                                            lstm_size=hidden_size, num_classes=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    Qs = tf.reduce_max(actions_logits*actions_labels, axis=1)
    loss = tf.reduce_mean(tf.square(Qs - targetQs))
    return actions_logits, final_state, loss

In [8]:
def model_opt(loss, learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    g_vars = [var for var in t_vars if var.name.startswith('generator')]

    # # Optimize
    # with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
    # #opt = tf.train.AdamOptimizer(learning_rate).minimize(loss, var_list=g_vars)

    #grads, _ = tf.clip_by_global_norm(t_list=tf.gradients(loss, g_vars), clip_norm=5) # usually around 1-5
    grads = tf.gradients(loss, g_vars)
    opt = tf.train.AdamOptimizer(learning_rate).apply_gradients(grads_and_vars=zip(grads, g_vars))

    return opt

In [9]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, learning_rate):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.targetQs, cell, self.initial_state = model_input(
            state_size=state_size, lstm_size=hidden_size)
        
        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.final_state, self.loss = model_loss(
            action_size=action_size, hidden_size=hidden_size, 
            states=self.states, actions=self.actions, 
            targetQs=self.targetQs, cell=cell, initial_state=self.initial_state)

        # Update the model: backward pass and backprop
        self.opt = model_opt(loss=self.loss, learning_rate=learning_rate)

## Experience replay

Reinforcement learning algorithms can have stability issues due to correlations between states. To reduce correlations when training, we can store the agent's experiences and later draw a random mini-batch of those experiences to train on. 

Here, we'll create a `Memory` object that will store our experiences, our transitions $<s, a, r, s'>$. This memory will have a maxmium capacity, so we can keep newer experiences in memory while getting rid of older experiences. Then, we'll sample a random mini-batch of transitions $<s, a, r, s'>$ and train on those.

Below, I've implemented a `Memory` object. If you're unfamiliar with `deque`, this is a double-ended queue. You can think of it like a tube open on both sides. You can put objects in either side of the tube. But if it's full, adding anything more will push an object out the other side. This is a great data structure to use for the memory buffer.

In [10]:
from collections import deque

class Memory():    
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size)
        self.states = deque(maxlen=max_size)
    def sample(self, batch_size):
        idx = np.random.choice(np.arange(len(self.buffer)), 
                               size=batch_size, 
                               replace=False)
        return [self.buffer[ii] for ii in idx], [self.states[ii] for ii in idx]

## Exploration - Exploitation

To learn about the environment and rules of the game, the agent needs to explore by taking random actions. We'll do this by choosing a random action with some probability $\epsilon$ (epsilon).  That is, with some probability $\epsilon$ the agent will make a random action and with probability $1 - \epsilon$, the agent will choose an action from $Q(s,a)$. This is called an **$\epsilon$-greedy policy**.


At first, the agent needs to do a lot of exploring. Later when it has learned more, the agent can favor choosing actions based on what it has learned. This is called _exploitation_. We'll set it up so the agent is more likely to explore early in training, then more likely to exploit later in training.

## Q-Learning training algorithm

Putting all this together, we can list out the algorithm we'll use to train the network. We'll train the network in _episodes_. One *episode* is one simulation of the game. For this game, the goal is to keep the pole upright for 195 frames. So we can start a new episode once meeting that goal. The game ends if the pole tilts over too far, or if the cart moves too far the left or right. When a game ends, we'll start a new episode. Now, to train the agent:

* Initialize the memory $D$
* Initialize the action-value network $Q$ with random weights
* **For** episode = 1, $M$ **do**
  * **For** $t$, $T$ **do**
     * With probability $\epsilon$ select a random action $a_t$, otherwise select $a_t = \mathrm{argmax}_a Q(s,a)$
     * Execute action $a_t$ in simulator and observe reward $r_{t+1}$ and new state $s_{t+1}$
     * Store transition $<s_t, a_t, r_{t+1}, s_{t+1}>$ in memory $D$
     * Sample random mini-batch from $D$: $<s_j, a_j, r_j, s'_j>$
     * Set $\hat{Q}_j = r_j$ if the episode ends at $j+1$, otherwise set $\hat{Q}_j = r_j + \gamma \max_{a'}{Q(s'_j, a')}$
     * Make a gradient descent step with loss $(\hat{Q}_j - Q(s_j, a_j))^2$
  * **endfor**
* **endfor**

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [11]:
# print('state:', np.array(states).shape[1], 
#       'action size: {}'.format((np.max(np.array(actions)) - np.min(np.array(actions)))+1))

In [12]:
# Network parameters
action_size = 2
state_size = 4
hidden_size = 64               # number of units in each Q-network hidden layer
learning_rate = 0.0001         # Q-network learning rate

# Memory parameters
memory_size = 128            # memory capacity - 1000 DQN
batch_size = 128             # experience mini-batch size - 20 DQN
gamma = 0.99                 # future reward discount

In [13]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, hidden_size=hidden_size, state_size=state_size, learning_rate=learning_rate)

# Init the memory
memory = Memory(max_size=batch_size)

(?, 4) (?, 64)
(1, ?, 64) (1, 64)
(1, ?, 64) (1, 64)
(?, 64)
(?, 2)


## Populate the memory (exprience memory)

Here I'm re-initializing the simulation and pre-populating the memory. The agent is taking random actions and storing the transitions in memory. This will help the agent with exploring the game.

In [14]:
state = env.reset()
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    memory.buffer.append([state, action, next_state, reward, float(done)])
    memory.states.append(np.zeros([1, hidden_size]))
    state = next_state
    if done is True:
        # Reseting the env/first state
        state = env.reset()

In [15]:
# # Training
# batch = memory.buffer
# states = np.array([each[0] for each in batch])
# actions = np.array([each[1] for each in batch])
# next_states = np.array([each[2] for each in batch])
# rewards = np.array([each[3] for each in batch])
# dones = np.array([each[4] for each in batch])

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [16]:
# initial_states = np.array(memory.states)
# initial_states.shape

In [ ]:
saver = tf.train.Saver()
episode_rewards_list, rewards_list, loss_list = [], [], []

# TF session for training
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    episode_reward = deque(maxlen=100) # 100 episodes average/running average/running mean/window
    
    # Training episodes/epochs
    for ep in range(11111):
        total_reward = 0
        loss_batch = []
        state = env.reset()
        initial_state = sess.run(model.initial_state)

        # Training steps/batches
        while True:
            action_logits, final_state = sess.run([model.actions_logits, model.final_state],
                                                  feed_dict = {model.states: state.reshape([1, -1]), 
                                                               model.initial_state: initial_state})
            action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done)])
            memory.states.append(initial_state)
            total_reward += reward
            state = next_state
            initial_state = final_state

            # Training
            batch = memory.buffer
            states = np.array([each[0] for each in batch])
            actions = np.array([each[1] for each in batch])
            next_states = np.array([each[2] for each in batch])
            rewards = np.array([each[3] for each in batch])
            dones = np.array([each[4] for each in batch])
            #initial_states = np.array(memory.states)
            initial_states = memory.states
            next_actions_logits = sess.run(model.actions_logits, 
                                           feed_dict = {model.states: next_states,
                                                        model.initial_state: initial_states[1]})
            nextQs = np.max(next_actions_logits, axis=1) * (1-dones)
            targetQs = rewards + (gamma * nextQs)
            loss, _ = sess.run([model.loss, model.opt], feed_dict = {model.states: states, 
                                                                     model.actions: actions,
                                                                     model.targetQs: targetQs,
                                                                     model.initial_state: initial_states[0]})
            # End of training
            loss_batch.append(loss)
            if done is True:
                break
                
        # Outputing: priting out/Potting
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'loss:{:.4f}'.format(np.mean(loss_batch)))
        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        loss_list.append([ep, np.mean(loss_batch)])
        # Break episode/epoch loop
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:52.0000 R:52.0000 loss:1.1622
Episode:1 meanR:45.5000 R:39.0000 loss:0.9973
Episode:2 meanR:41.3333 R:33.0000 loss:1.0668
Episode:3 meanR:41.5000 R:42.0000 loss:1.0128
Episode:4 meanR:41.0000 R:39.0000 loss:1.3224
Episode:5 meanR:37.6667 R:21.0000 loss:1.6030
Episode:6 meanR:34.7143 R:17.0000 loss:1.8615
Episode:7 meanR:32.6250 R:18.0000 loss:2.1240
Episode:8 meanR:31.2222 R:20.0000 loss:2.4263
Episode:9 meanR:29.7000 R:16.0000 loss:2.7760
Episode:10 meanR:28.8182 R:20.0000 loss:2.9228
Episode:11 meanR:27.9167 R:18.0000 loss:3.1931
Episode:12 meanR:27.6154 R:24.0000 loss:3.2269
Episode:13 meanR:27.4286 R:25.0000 loss:3.1949
Episode:14 meanR:27.4667 R:28.0000 loss:3.2389
Episode:15 meanR:26.9375 R:19.0000 loss:3.3012
Episode:16 meanR:26.5882 R:21.0000 loss:3.4123
Episode:17 meanR:26.4444 R:24.0000 loss:3.4982
Episode:18 meanR:26.3684 R:25.0000 loss:3.6283
Episode:19 meanR:26.3000 R:25.0000 loss:3.6873
Episode:20 meanR:26.2857 R:26.0000 loss:3.8464
Episode:21 meanR:26.000

Episode:173 meanR:37.2800 R:102.0000 loss:0.7342
Episode:174 meanR:37.5100 R:34.0000 loss:1.7905
Episode:175 meanR:37.7000 R:31.0000 loss:9.7223
Episode:176 meanR:37.8400 R:27.0000 loss:16.4098
Episode:177 meanR:38.4700 R:78.0000 loss:19.2473
Episode:178 meanR:38.5800 R:24.0000 loss:13.5344
Episode:179 meanR:38.6300 R:20.0000 loss:15.1545
Episode:180 meanR:38.6800 R:18.0000 loss:17.6511
Episode:181 meanR:38.7700 R:24.0000 loss:15.7806
Episode:182 meanR:39.7100 R:108.0000 loss:72.1386
Episode:183 meanR:40.7300 R:120.0000 loss:9.9799
Episode:184 meanR:40.8500 R:31.0000 loss:9.6109
Episode:185 meanR:41.0100 R:33.0000 loss:4.7333
Episode:186 meanR:41.1900 R:31.0000 loss:8.5522
Episode:187 meanR:42.7900 R:176.0000 loss:8.4791
Episode:188 meanR:43.7900 R:117.0000 loss:9.4129
Episode:189 meanR:43.8500 R:20.0000 loss:12.7997
Episode:190 meanR:43.8300 R:18.0000 loss:15.7790
Episode:191 meanR:43.9000 R:25.0000 loss:18.9757
Episode:192 meanR:44.9800 R:130.0000 loss:13.3513
Episode:193 meanR:45.95

Episode:339 meanR:289.9600 R:500.0000 loss:15.9674
Episode:340 meanR:294.1800 R:500.0000 loss:15.6195
Episode:341 meanR:298.5100 R:500.0000 loss:15.6869
Episode:342 meanR:300.6600 R:272.0000 loss:28.6322
Episode:343 meanR:305.0600 R:487.0000 loss:15.3149
Episode:344 meanR:307.9000 R:339.0000 loss:22.5444
Episode:345 meanR:309.2300 R:192.0000 loss:27.9462
Episode:346 meanR:310.2800 R:159.0000 loss:19.8253
Episode:347 meanR:310.9000 R:98.0000 loss:20.1440
Episode:348 meanR:310.9900 R:54.0000 loss:68.2759
Episode:349 meanR:311.2400 R:51.0000 loss:66.1013
Episode:350 meanR:315.8100 R:500.0000 loss:7.2205
Episode:351 meanR:320.3200 R:500.0000 loss:4.7770
Episode:352 meanR:323.0600 R:307.0000 loss:25.1335
Episode:353 meanR:326.6400 R:500.0000 loss:6.3606
Episode:354 meanR:327.8500 R:500.0000 loss:14.5608
Episode:355 meanR:328.7300 R:500.0000 loss:6.2355
Episode:356 meanR:328.0000 R:83.0000 loss:55.3152
Episode:357 meanR:331.6600 R:500.0000 loss:23.0249
Episode:358 meanR:335.3900 R:500.0000 l

Episode:500 meanR:339.3000 R:11.0000 loss:340.3174
Episode:501 meanR:334.4000 R:10.0000 loss:341.3881
Episode:502 meanR:329.4900 R:9.0000 loss:336.7712
Episode:503 meanR:324.5800 R:9.0000 loss:343.1965
Episode:504 meanR:319.6900 R:11.0000 loss:347.2753
Episode:505 meanR:318.0200 R:10.0000 loss:347.4333
Episode:506 meanR:313.1100 R:9.0000 loss:346.4355
Episode:507 meanR:308.2000 R:9.0000 loss:347.8940
Episode:508 meanR:306.5200 R:9.0000 loss:350.7670
Episode:509 meanR:301.6100 R:9.0000 loss:351.6598
Episode:510 meanR:296.7100 R:10.0000 loss:347.6264
Episode:511 meanR:291.8100 R:10.0000 loss:344.3522
Episode:512 meanR:286.8900 R:8.0000 loss:336.9310
Episode:513 meanR:281.9800 R:9.0000 loss:338.4356
Episode:514 meanR:279.3000 R:10.0000 loss:330.6525
Episode:515 meanR:274.3900 R:9.0000 loss:320.4992
Episode:516 meanR:269.4800 R:9.0000 loss:310.7948
Episode:517 meanR:264.5700 R:9.0000 loss:299.8535
Episode:518 meanR:259.6600 R:9.0000 loss:291.5416
Episode:519 meanR:254.7500 R:9.0000 loss:28

Episode:666 meanR:17.8200 R:20.0000 loss:10.6096
Episode:667 meanR:17.9100 R:22.0000 loss:10.2937
Episode:668 meanR:17.9500 R:18.0000 loss:11.0787
Episode:669 meanR:18.0500 R:23.0000 loss:10.6568
Episode:670 meanR:18.1300 R:23.0000 loss:10.6074
Episode:671 meanR:18.2000 R:20.0000 loss:10.5958
Episode:672 meanR:18.2500 R:18.0000 loss:10.6981
Episode:673 meanR:18.3400 R:21.0000 loss:10.1802
Episode:674 meanR:18.4200 R:20.0000 loss:10.5114
Episode:675 meanR:18.5000 R:21.0000 loss:10.1636
Episode:676 meanR:18.6000 R:22.0000 loss:9.7167
Episode:677 meanR:18.6700 R:22.0000 loss:10.2270
Episode:678 meanR:18.7600 R:21.0000 loss:9.9439
Episode:679 meanR:18.7900 R:19.0000 loss:9.6087
Episode:680 meanR:18.8700 R:22.0000 loss:9.3588
Episode:681 meanR:18.9300 R:19.0000 loss:9.0355
Episode:682 meanR:19.0000 R:23.0000 loss:8.6183
Episode:683 meanR:19.0700 R:19.0000 loss:8.7489
Episode:684 meanR:19.1300 R:19.0000 loss:8.9008
Episode:685 meanR:19.2100 R:20.0000 loss:8.6647
Episode:686 meanR:19.2800 R:2

...

Episode:8481 meanR:312.7900 R:500.0000 loss:15.1326
Episode:8482 meanR:311.4800 R:10.0000 loss:73.5051
Episode:8483 meanR:309.8800 R:10.0000 loss:125.0860
Episode:8484 meanR:308.8600 R:16.0000 loss:152.3776
Episode:8485 meanR:313.7500 R:500.0000 loss:18.0431
Episode:8486 meanR:313.8800 R:30.0000 loss:65.0279
Episode:8487 meanR:314.9100 R:212.0000 loss:35.8863
Episode:8488 meanR:314.9100 R:500.0000 loss:8.7263
Episode:8489 meanR:318.3400 R:500.0000 loss:14.3613
Episode:8490 meanR:313.8000 R:46.0000 loss:69.3001
Episode:8491 meanR:308.9400 R:14.0000 loss:128.6534
Episode:8492 meanR:304.3700 R:43.0000 loss:141.8902
Episode:8493 meanR:301.3600 R:199.0000 loss:57.8576
Episode:8494 meanR:298.1200 R:176.0000 loss:2.6646
Episode:8495 meanR:294.7600 R:164.0000 loss:2.0935
Episode:8496 meanR:291.6500 R:189.0000 loss:2.1035
Episode:8497 meanR:291.7000 R:146.0000 loss:1.4889
Episode:8498 meanR:292.3900 R:145.0000 loss:1.2543
Episode:8499 meanR:292.8600 R:139.0000 loss:1.2071
Episode:8500 meanR:290

Episode:8640 meanR:434.3700 R:500.0000 loss:17.0680
Episode:8641 meanR:434.3700 R:500.0000 loss:17.4789
Episode:8642 meanR:434.3700 R:500.0000 loss:16.5497
Episode:8643 meanR:434.3700 R:500.0000 loss:16.1413
Episode:8644 meanR:429.4800 R:11.0000 loss:72.4531
Episode:8645 meanR:424.6100 R:13.0000 loss:102.3807
Episode:8646 meanR:423.5600 R:395.0000 loss:18.7431
Episode:8647 meanR:422.0600 R:350.0000 loss:5.9357
Episode:8648 meanR:420.1600 R:310.0000 loss:3.6771
Episode:8649 meanR:420.1600 R:500.0000 loss:6.1022
Episode:8650 meanR:420.1600 R:500.0000 loss:18.2182
Episode:8651 meanR:420.1600 R:500.0000 loss:18.7506
Episode:8652 meanR:425.0500 R:500.0000 loss:17.6776
Episode:8653 meanR:425.0500 R:500.0000 loss:17.3089
Episode:8654 meanR:425.0500 R:500.0000 loss:16.8934
Episode:8655 meanR:425.0500 R:500.0000 loss:16.9767
Episode:8656 meanR:425.0500 R:500.0000 loss:16.7817
Episode:8657 meanR:425.0500 R:500.0000 loss:16.5145
Episode:8658 meanR:420.2000 R:15.0000 loss:72.8071
Episode:8659 mean

Episode:8799 meanR:344.5200 R:500.0000 loss:17.5034
Episode:8800 meanR:344.5200 R:500.0000 loss:15.0421
Episode:8801 meanR:344.5200 R:500.0000 loss:12.2530
Episode:8802 meanR:344.5200 R:500.0000 loss:15.8906
Episode:8803 meanR:347.1300 R:500.0000 loss:18.5358
Episode:8804 meanR:352.0100 R:500.0000 loss:15.1739
Episode:8805 meanR:356.8900 R:500.0000 loss:14.4861
Episode:8806 meanR:361.7700 R:500.0000 loss:15.7965
Episode:8807 meanR:366.6600 R:500.0000 loss:9.6750
Episode:8808 meanR:371.5300 R:500.0000 loss:15.2472
Episode:8809 meanR:371.9700 R:500.0000 loss:18.6634
Episode:8810 meanR:371.9700 R:500.0000 loss:16.5998
Episode:8811 meanR:371.9700 R:500.0000 loss:17.3374
Episode:8812 meanR:371.9700 R:500.0000 loss:16.2014
Episode:8813 meanR:367.1000 R:13.0000 loss:74.8373
Episode:8814 meanR:366.3600 R:31.0000 loss:122.4213
Episode:8815 meanR:366.4800 R:97.0000 loss:103.2635
Episode:8816 meanR:370.2000 R:500.0000 loss:4.9033
Episode:8817 meanR:373.8200 R:500.0000 loss:15.8207
Episode:8818 me

Episode:8957 meanR:431.8000 R:500.0000 loss:19.4228
Episode:8958 meanR:431.8000 R:500.0000 loss:19.6477
Episode:8959 meanR:431.8000 R:500.0000 loss:13.0991
Episode:8960 meanR:431.8000 R:500.0000 loss:12.8124
Episode:8961 meanR:431.8000 R:500.0000 loss:12.0254
Episode:8962 meanR:431.8000 R:500.0000 loss:13.3981
Episode:8963 meanR:431.8000 R:500.0000 loss:12.3087
Episode:8964 meanR:431.8000 R:500.0000 loss:14.8183
Episode:8965 meanR:431.8000 R:500.0000 loss:14.3636
Episode:8966 meanR:431.8000 R:500.0000 loss:14.9198
Episode:8967 meanR:431.8000 R:500.0000 loss:16.7380
Episode:8968 meanR:431.8000 R:500.0000 loss:13.4966
Episode:8969 meanR:431.8000 R:500.0000 loss:9.1049
Episode:8970 meanR:431.8000 R:500.0000 loss:13.8467
Episode:8971 meanR:431.8000 R:500.0000 loss:12.2850
Episode:8972 meanR:431.8000 R:500.0000 loss:15.2456
Episode:8973 meanR:431.8000 R:500.0000 loss:13.7061
Episode:8974 meanR:431.8000 R:500.0000 loss:15.0676
Episode:8975 meanR:431.8000 R:500.0000 loss:15.0047
Episode:8976 

Episode:9115 meanR:435.3000 R:500.0000 loss:18.5768
Episode:9116 meanR:435.3000 R:500.0000 loss:16.5022
Episode:9117 meanR:435.3000 R:500.0000 loss:14.8494
Episode:9118 meanR:435.3000 R:500.0000 loss:15.4706
Episode:9119 meanR:435.3000 R:500.0000 loss:15.2699
Episode:9120 meanR:435.3000 R:500.0000 loss:15.3586
Episode:9121 meanR:435.3000 R:500.0000 loss:16.0633
Episode:9122 meanR:435.3000 R:500.0000 loss:15.1276
Episode:9123 meanR:435.3000 R:500.0000 loss:14.3538
Episode:9124 meanR:435.3000 R:500.0000 loss:16.9442
Episode:9125 meanR:435.3000 R:500.0000 loss:16.8663
Episode:9126 meanR:435.3000 R:500.0000 loss:16.7988
Episode:9127 meanR:435.3000 R:500.0000 loss:16.4024
Episode:9128 meanR:435.3000 R:500.0000 loss:15.2384
Episode:9129 meanR:435.3000 R:500.0000 loss:15.7307


# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(loss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Average losses')

## Testing

Let's checkout how our trained agent plays the game.

In [38]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episode/epoch
    for _ in range(10):
        total_reward = 0
        state = env.reset()
        initial_state = sess.run(model.initial_state) # Qs or current batch or states[:-1]
        
        # Steps/batches
        while True:
            env.render()
            action_logits, initial_state = sess.run([model.actions_logits, model.final_state],
                                                    feed_dict = {model.states: state.reshape([1, -1]), 
                                                                 model.initial_state: initial_state})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                break
        # At the end of each episode
        print('total_reward:{}'.format(total_reward))

# Close the env
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward:500.0
total_reward:500.0
total_reward:500.0
total_reward:500.0
total_reward:500.0
total_reward:500.0
total_reward:500.0
total_reward:500.0
total_reward:500.0
total_reward:500.0


## Extending this

So, Cart-Pole is a pretty simple game. However, the same model can be used to train an agent to play something much more complicated like Pong or Space Invaders. Instead of a state like we're using here though, you'd want to use convolutional layers to get the state from the screen images.

![Deep Q-Learning Atari](assets/atari-network.png)

I'll leave it as a challenge for you to use deep Q-learning to train an agent to play Atari games. Here's the original paper which will get you started: http://www.davidqiu.com:8888/research/nature14236.pdf.